# ENTRENAMIENTO MODELO DE SUGERENCIAS DE TICKETS
* Carga el dataset procesado
* Construye los modelos de cada variable dependiente
* Divide el dataset en entrenamiento y validación
* Entrena los modelos
* Evalúa su precisión

# Importe de librerías

In [2]:
import pandas as pd
import numpy as np

In [3]:
from IPython.core.display import display, HTML
import warnings
warnings.filterwarnings('ignore')
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 800)

In [4]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
import gensim
import pyLDAvis.gensim_models
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.models import LdaModel

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import ToktokTokenizer
import nltk

C:\ProgramData\Anaconda3\envs\T\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [5]:
import json, re
from joblib import dump,load

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMClassifier

# Modelado FAILURECODE

In [7]:
df_base=pd.read_csv('DATOS/base_parcial.csv').set_index('TICKETID')

In [7]:
X_columns=df_base.columns.values.tolist()

In [8]:
y_columns=['FAILURECODE','REMEDY','CAUSE','PROBLEM']

In [9]:
X_columns

['CL_INC_IMPACTO_0',
 'CL_INC_IMPACTO_1',
 'CL_INC_IMPACTO_2',
 'CL_INC_IMPACTO_3',
 'CL_INC_IMPACTO_4',
 'DESCRIPTION_0',
 'DESCRIPTION_1',
 'DESCRIPTION_2',
 'DESCRIPTION_3',
 'DESCRIPTION_4',
 'INCMEXCLUSION_0',
 'INCMEXCLUSION_1',
 'INCMEXCLUSION_2',
 'INCMEXCLUSION_3',
 'INCMEXCLUSION_4',
 'INCSOLUCION_0',
 'INCSOLUCION_1',
 'INCSOLUCION_2',
 'INCSOLUCION_3',
 'INCSOLUCION_4',
 'IMPACT',
 'INCCARGAPRIORIDAD',
 'INTERNALPRIORITY',
 'OUTAGEDURATION',
 'URGENCY',
 'DIAGNOSTICADO',
 'HASACTIVITY',
 'HASLD',
 'INCEXCLUIR',
 'INCEXCLUIR_ENLD',
 'INC_ALARMA',
 'MUN100',
 'SRDATOS',
 'SRINTERNET',
 'SRTV',
 'SRVOZ',
 'HASCHILDREN',
 'USECLASSINDESC',
 'ASSIGNEDOWNERGROUP_AMDOCS',
 'ASSIGNEDOWNERGROUP_AMDOCS_OPTIMIZACION',
 'ASSIGNEDOWNERGROUP_AMDOCS_TRIAGE_2G',
 'ASSIGNEDOWNERGROUP_AMDOCS_TRIAGE_4G',
 'ASSIGNEDOWNERGROUP_BACKTXMW_SATEL_PERFORMANCE',
 'ASSIGNEDOWNERGROUP_BACK_AUT_PRUEBA',
 'ASSIGNEDOWNERGROUP_BODATOS',
 'ASSIGNEDOWNERGROUP_BOE_FIJA',
 'ASSIGNEDOWNERGROUP_BOE_MOVIL',
 'ASSI

In [10]:
len(X_columns)

1376

In [11]:
type(X_columns)

list

In [12]:
X_columns.remove('FAILURECODE')
X_columns.remove('REMEDY')
X_columns.remove('CAUSE')
X_columns.remove('PROBLEM')

In [13]:
X_Var=df_base[X_columns]

In [14]:
y_Var=df_base[y_columns]

In [ ]:
import gc
del df_base
gc.collect()

In [ ]:
import gc
del X_train
del X_test
del y_train
del y_test
gc.collect()

In [ ]:
len(X_Var.columns)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_Var, y_Var['FAILURECODE'], test_size = 0.2, random_state = 86)

In [ ]:
modelLGBM=LGBMClassifier(n_estimators=500, learning_rate=0.01, reg_alpha=1, reg_lambda=3)

In [ ]:
modelLGBM.fit(X_train, y_train)

In [ ]:
#Guardar Modelo
ruta_objeto= f"OBJETOS/modelo_failureCode.pkl"
dump(modelLGBM, ruta_objeto)

In [ ]:
#Probabilidades, no es necesario en entrenamiento
#pred_test_proba=modelLGBM.predict_proba(X_test)[:,1]

In [ ]:
pred_test=modelLGBM.predict(X_test)

In [ ]:
# Exactitud

In [ ]:
pred_test

In [ ]:
len(pred_test)

In [ ]:
len(pred_test_proba)

In [ ]:
df_test=y_test.to_frame('real')
df_test

In [ ]:
df_test['prediccion']=pred_test
df_test

In [ ]:
df_test['resultado']=np.where(df_test['prediccion']==df_test['real'],1,0)

In [ ]:
df_test['resultado'].sum()

In [ ]:
df_test['resultado'].sum()/len(df_test)

# Modelo Problem

In [ ]:
del X_train
del X_test
del y_train
del y_test
gc.collect()

In [7]:
df_base=pd.read_csv('OBJETOS/base_parcial.csv').set_index('TICKETID')

In [9]:
#eliminar los problem null
len(df_base[df_base['PROBLEM'].isnull()])

0

In [10]:
len(df_base)

1519062

In [11]:
df_base=df_base[df_base['PROBLEM'].notnull()]

In [12]:
X_columns=df_base.columns.values.tolist()
y_columns=['FAILURECODE','REMEDY','CAUSE','PROBLEM']

In [13]:
X_columns.remove('FAILURECODE')
X_columns.remove('REMEDY')
X_columns.remove('CAUSE')
X_columns.remove('PROBLEM')

In [14]:
X_Var=df_base[X_columns]
y_Var=df_base[y_columns]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_Var, y_Var['PROBLEM'], test_size = 0.2, random_state = 86)

In [16]:
import gc
del df_base
gc.collect()

0

In [17]:
modelLGBM_problem=LGBMClassifier(n_estimators=500, learning_rate=0.01, reg_alpha=1, reg_lambda=3)

In [18]:
modelLGBM_problem.fit(X_train, y_train)

LGBMClassifier(learning_rate=0.01, n_estimators=500, reg_alpha=1, reg_lambda=3)

In [19]:
#Guardar modelo
ruta_objeto= f"OBJETOS/modelo_problem.pkl"
dump(modelLGBM_problem, ruta_objeto)

['modeloEntrenado/modelo_problem.pkl']

In [6]:
modelLGBM_problem=load(f"OBJETOS/modelo_problem.pkl")

In [ ]:
#Probabilidades, no es necesario en entrenamiento
#pred_test_proba=modelLGBM_problem.predict_proba(X_test)[:,1]

In [17]:
pred_test=modelLGBM_problem.predict(X_test)

In [18]:
df_test=y_test.to_frame('real')
df_test

,real
TICKETID,
INC3947438,SIN FALLA O&M ACCESO
INC1720191,PENDIENTE DOCUMENTACION DE CAUSAS DE CIERRE
INC1644694,PENDIENTE DOCUMENTACION DE CAUSAS DE CIERRE
INC1004326,SIT_PARC_2G
INC2249391,PRUEBA_DATOS
...,...
INC1796450,NOT_ENERGIA
INC4651848,SIN FALLA O&M ACCESO
INC3043724,OTRO


In [19]:
df_test['prediccion']=pred_test
df_test

,real,prediccion
TICKETID,,
INC3947438,SIN FALLA O&M ACCESO,SIN FALLA O&M ACCESO
INC1720191,PENDIENTE DOCUMENTACION DE CAUSAS DE CIERRE,PENDIENTE DOCUMENTACION DE CAUSAS DE CIERRE
INC1644694,PENDIENTE DOCUMENTACION DE CAUSAS DE CIERRE,PENDIENTE DOCUMENTACION DE CAUSAS DE CIERRE
INC1004326,SIT_PARC_2G,SIT_PARC_2G
INC2249391,PRUEBA_DATOS,PRUEBA_DATOS
...,...,...
INC1796450,NOT_ENERGIA,NOT_ENERGIA
INC4651848,SIN FALLA O&M ACCESO,AMPLIFICADOR
INC3043724,OTRO,CMTS


In [20]:
df_test['resultado']=np.where(df_test['prediccion']==df_test['real'],1,0)

In [21]:
df_test['resultado'].sum()

245265

In [22]:
df_test['resultado'].sum()/len(df_test)

0.807289352331862

# Modelo Causa

In [23]:
del X_train
del X_test
del y_train
del y_test
gc.collect()

0

In [24]:
df_base=pd.read_csv('OBJETOS/base_parcial.csv').set_index('TICKETID')

In [8]:
#eliminar los problem null
len(df_base[df_base['CAUSE'].isnull()])

0

In [9]:
len(df_base)

1519062

In [10]:
len(df_base)

1519062

In [11]:
df_base=df_base[df_base['CAUSE'].notnull()]

In [12]:
X_columns=df_base.columns.values.tolist()
y_columns=['FAILURECODE','REMEDY','CAUSE','PROBLEM']

In [13]:
X_columns.remove('FAILURECODE')
X_columns.remove('REMEDY')
X_columns.remove('CAUSE')
X_columns.remove('PROBLEM')

In [14]:
X_Var=df_base[X_columns]
y_Var=df_base[y_columns]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_Var, y_Var['CAUSE'], test_size = 0.2, random_state = 86)

In [16]:
import gc
del df_base
gc.collect()

0

In [17]:
modelLGBM_cause=LGBMClassifier(n_estimators=500, learning_rate=0.01, reg_alpha=1, reg_lambda=3)

In [ ]:
modelLGBM_cause.fit(X_train, y_train)

In [ ]:
#Guardar modelo
ruta_objeto= f"OBJETOS/modelo_cause.pkl"
dump(modelLGBM_cause, ruta_objeto)

In [ ]:
#Probabilidades, no es necesario en entrenamiento
#pred_test_proba=modelLGBM_cause.predict_proba(X_test)[:,1]

In [ ]:
pred_test=modelLGBM_cause.predict(X_test)

In [ ]:
df_test=y_test.to_frame('real')
df_test

In [ ]:
df_test['prediccion']=pred_test
df_test

In [ ]:
df_test['resultado']=np.where(df_test['prediccion']==df_test['real'],1,0)

In [ ]:
df_test['resultado'].sum()

In [ ]:
df_test['resultado'].sum()/len(df_test)

# Modelo Remedy

In [ ]:
del X_train
del X_test
del y_train
del y_test
gc.collect()

In [ ]:
df_base=pd.read_csv('OBJETOS/base_parcial.csv').set_index('TICKETID')

In [ ]:
#eliminar los remedy null
len(df_base[df_base['REMEDY'].isnull()])

In [ ]:
len(df_base)

In [ ]:
len(df_base)

In [ ]:
df_base=df_base[df_base['REMEDY'].notnull()]

In [ ]:
X_columns=df_base.columns.values.tolist()
y_columns=['FAILURECODE','REMEDY','CAUSE','PROBLEM']

In [ ]:
X_columns.remove('FAILURECODE')
X_columns.remove('REMEDY')
X_columns.remove('CAUSE')
X_columns.remove('PROBLEM')

In [ ]:
X_Var=df_base[X_columns]
y_Var=df_base[y_columns]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_Var, y_Var['REMEDY'], test_size = 0.2, random_state = 86)

In [ ]:
import gc
del df_base
gc.collect()

In [ ]:
modelLGBM_remedy=LGBMClassifier(n_estimators=500, learning_rate=0.01, reg_alpha=1, reg_lambda=3)

In [ ]:
modelLGBM_remedy.fit(X_train, y_train)

In [ ]:
#Guardar modelo
ruta_objeto= f"OBJETOS/modelo_remedy.pkl"
dump(modelLGBM_remedy, ruta_objeto)

In [ ]:
#Probabilidades, no es necesario en entrenamiento
#pred_test_proba=modelLGBM_remedy.predict_proba(X_test)[:,1]

In [ ]:
pred_test=modelLGBM_remedy.predict(X_test)

In [ ]:
df_test=y_test.to_frame('real')
df_test

In [ ]:
df_test['prediccion']=pred_test
df_test

In [ ]:
df_test['resultado']=np.where(df_test['prediccion']==df_test['real'],1,0)

In [ ]:
df_test['resultado'].sum()

In [ ]:
df_test['resultado'].sum()/len(df_test)

## Manejo memoria

In [ ]:
import numpy as np
import pandas as pd
import sys
def show_mem_usage():
    '''Displays memory usage from inspection
    of global variables in this notebook'''
    gl = sys._getframe(1).f_globals
    vars= {}
    for k,v in list(gl.items()):
        # for pandas dataframes
        if hasattr(v, 'memory_usage'):
            mem = v.memory_usage(deep=True)
            if not np.isscalar(mem):
                mem = mem.sum()
            vars.setdefault(id(v),[mem]).append(k)
        # work around for a bug
        ##elif isinstance(v,pd.Panel):
        ##    v = v.values
        vars.setdefault(id(v),[sys.getsizeof(v)]).append(k)
    total = 0
    for k,(value,*names) in vars.items():
        if value>1e6:
            print(names,"%.3fMB"%(value/1e6))
        total += value
    print("%.3fMB"%(total/1e6))

In [ ]:
show_mem_usage()

In [ ]:
del aux_df
gc.collect()